# Advent of Code 2021 - Day 5

In [13]:
data class Point(val x: Int, val y: Int)
data class LineSegment(val from: Point, val to: Point)

In [14]:
import java.io.File
import java.util.Scanner

val regex = Regex("""(\d+),(\d+) -> (\d+),(\d+)""")

val lineSegments: List<LineSegment> = File("Day5.input.txt")
    .bufferedReader()
    .readLines()
    .map { regex.find(it)!!.destructured.toList() }
    .map { it.map(String::toInt) }
    .map { (x1, y1, x2, y2) -> LineSegment(from = Point(x = x1, y = y1), to = Point(x = x2, y = y2)) }

## Part 1

Calculate the number of points at least two lines overlap. Consider only horizontal and vertical lines.

In [15]:
fun LineSegment.points(): Set<Point> {
    val x = from.x - to.x
    val y = from.y - to.y
    val length = max(abs(x), abs(y))
    val dx = x / length * -1
    val dy = y / length * -1
    
    return (1..length).fold(mutableListOf(from)) { acc, index -> 
        acc.apply { add(Point(x = from.x + (dx * index), y = from.y + (dy * index))) }
    }.toSet()
}

fun LineSegment.intersect(other: LineSegment): Set<Point> = points().intersect(other.points())

fun List<LineSegment>.intersections(): Set<Point> =
    fold(mutableSetOf<Point>()) { acc1, line1 ->
        fold(acc1) { acc2, line2 ->
            if (line1 != line2) acc2.apply { addAll(line1.intersect(line2)) } else acc2
        }
    }

lineSegments
    .filter { it.from.x == it.to.x || it.from.y == it.to.y }
    .intersections()
    .size

4745

### Notes

There's probably a mathematical solution to this, but I don't know it. A similar algorithm I would consider as well is to put all of the `LineSegment::points` into a grid, and then count the number of times a cell was updated (2+ would be an intersection). However, I decided to `intersect` since I felt like it was neat way to solve it (and it uses `Set::intersect` and we're looking for line intersections, so that's fun).

Basically, figure out the discrete points for each `LineSegment` and then intersect all of them. I suspect you could probably do some optimizations by omitting `LineSegment::intersect` calls that you can quickly determine will never intersect. Maybe with a bounding box?

## Part 2

Same as part 1 but consider all lines (diagonals).

In [16]:
lineSegments
    .intersections()
    .size

18442

### Notes

The `LineSegment::points` function already calculated diagonals, so we just didn't filter them out this time.